In [64]:
using DataFrames, CSV
using JLD
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions
using XLSX

In [63]:
import Pkg
Pkg.add("XLSX")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed XLSX ──── v0.8.4
   Installed ZipFile ─ v0.10.1
   Installed EzXML ─── v1.1.0
    Updating `~/.julia/environments/v1.8/Project.toml`
  [fdbf4ff8] + XLSX v0.8.4
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [8f5d6c58] + EzXML v1.1.0
  [fdbf4ff8] + XLSX v0.8.4
  [a5390f91] + ZipFile v0.10.1
Precompiling project...
  ✓ ZipFile
  ✓ EzXML
  ✓ XLSX
  3 dependencies successfully precompiled in 5 seconds. 211 already precompiled.


In [76]:
liverpool_data = CSV.read("Liverpool_FIFA_22.csv", DataFrame)
delete!(liverpool_data, [19, 21, 23, 28, 29, 30, 31, 32, 33])
# n_players = nrow(liverpool_data)

Row,Column1,short_name,player_positions,overall,age,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,max_games
,Int64,String31,String15,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,15,V. van Dijk,CB,89,29,73,73,73,67,69,69,69,67,73,73,73,73,78,78,78,73,82,87,87,87,82,84,89,89,89,84,23,35
2,16,S. Mané,LW,89,29,89,89,89,87,88,88,88,87,89,89,89,89,83,83,83,89,71,68,68,68,71,68,62,62,62,68,22,35
3,17,M. Salah,RW,89,29,88,88,88,88,88,88,88,88,89,89,89,89,85,85,85,89,74,71,71,71,74,70,61,61,61,70,25,35
4,20,Alisson,GK,89,28,40,40,40,35,38,38,38,35,43,43,43,41,43,43,43,41,34,38,38,38,34,33,34,34,34,33,90,35
5,42,A. Robertson,LB,87,27,75,75,75,79,77,77,77,79,82,82,82,84,84,84,84,84,88,85,85,85,88,87,82,82,82,87,21,35
6,44,T. Alexander-Arnold,RB,87,22,78,78,78,80,79,79,79,80,84,84,84,85,87,87,87,85,87,86,86,86,87,86,81,81,81,86,22,35
7,51,Thiago,"CM, CDM",86,30,77,77,77,81,80,80,80,81,86,86,86,85,86,86,86,85,80,83,83,83,80,78,74,74,74,78,21,25
8,61,Fabinho,"CDM, CB",86,27,77,77,77,75,75,75,75,75,79,79,79,80,84,84,84,80,85,87,87,87,85,85,87,87,87,85,21,30
9,85,Roberto Firmino,CF,85,29,84,84,84,83,83,83,83,83,85,85,85,85,85,85,85,85,75,75,75,75,75,72,70,70,70,72,20,35


In [25]:
all_teams = CSV.read("Team_Ratings.csv", DataFrame)
n_positions = 27
C = CSV.read("fixture_list.csv", DataFrame)
n_games = nrow(C)

38

In [26]:
GK = "gk"
defenders = ["lwb", "rwb", "lb", "lcb", "cb", "rcb", "rb"]
midfielders = ["lam", "cam", "ram", "lm", "lcm", "cm", "rcm", "rm", "ldm", "cdm", "rdm"]
forwards = ["ls", "st", "rs", "lw", "lf", "cf", "rf", "rw"]
center_backs = ["lcb", "cb", "rcb"]
right_backs = ["rwb", "rb"]
left_backs = ["lwb", "lb"]
cdms = ["ldm", "cdm", "rdm"]
cms = ["lcm", "cm", "rcm"]
cams = ["lam", "cam", "ram"]
left_wingers = ["lw", "lm"]
right_wingers = ["rw", "rm"]
strikers = ["ls", "st", "rs", "lf", "cf", "rf"]

6-element Vector{String}:
 "ls"
 "st"
 "rs"
 "lf"
 "cf"
 "rf"

In [27]:
player_pos = ["ls", "st", "rs", "lw", "lf", "cf", "rf", "rw", "lam", "cam", "ram", "lm", "lcm", "cm", "rcm", "rm", "lwb", "ldm", "cdm", "rdm", "rwb", "lb", "lcb", "cb", "rcb", "rb", "gk"]
pos_to_ind = Dict()
ind_to_pos = Dict()
index = 1
for pos in player_pos
    pos_to_ind[pos] = index
    ind_to_pos[index] = pos
    index += 1
end
pos_to_ind["st"]

2

In [100]:
defensive_pos = Set(["lb", "lcb", "cb", "rcb", "rb", "ldm", "cdm", "rdm"])
central_pos = Set(["ldm", "cdm", "rdm", "lcm", "cm", "rcm", "lam", "cam", "ram"])
forward_pos = Set(forwards)
wing_pos = Set(["lwb", "rwb", "lm", "rm", "lw", "rw"])
wingback_pos = Set(["lwb", "rwb", "lb", "rb"])

starting_11 = Set([1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12])
union(wingback_pos, wing_pos)

Set{String} with 8 elements:
  "rm"
  "rwb"
  "lb"
  "rw"
  "lw"
  "lm"
  "lwb"
  "rb"

In [77]:
function acceptable_positions(player_orig)
    acceptable_pos = Set()
    for k in player_orig
        push!(acceptable_pos, lowercase(k))
    end
    return acceptable_pos
end

R = zeros(Int64, (n_players, n_positions))
for i in 1:n_players
    player_orig = split(liverpool_data[i, "player_positions"], ", ")
    acceptable_pos = acceptable_positions(player_orig)
    for pos in player_pos
        if pos in acceptable_pos
            R[i, pos_to_ind[pos]] = liverpool_data[i, pos]
        else
            R[i, pos_to_ind[pos]] = -100
        end
    end
end
R[5, :]

27-element Vector{Int64}:
 -100
 -100
 -100
 -100
 -100
 -100
 -100
 -100
 -100
 -100
 -100
 -100
 -100
    ⋮
 -100
 -100
 -100
 -100
 -100
 -100
   87
 -100
 -100
 -100
 -100
 -100

In [222]:
pos_discount = 0.99
function penalty(player_number, game_number, x)
    if game_number == 1
        return 1
    end
    if sum(x[player_number, game_number-1, k] for k in 1:n_positions) == 0
        return 1
    end
    max_consecutive = 0
    for j in game_number-1:-1:max(1, game_number-6)
        if sum(x[player_number, j, k] for k in 1:n_positions) == 0
            max_consecutive += 0
        else
            max_consecutive += 1
        end
    end
    return pos_discount^max_consecutive
end

penalty (generic function with 1 method)

In [223]:
T_mult = 1.01

# model = Model(with_optimizer(Gurobi.Optimizer))
model = Model(Gurobi.Optimizer)

### Decision Variables ###

### x[i][j][k]. player i, game j, position k ###
@variable(model, x[i=1:n_players, j=1:n_games, k=1:n_positions], Bin)

### z variable, for whether we go for win or not ###
@variable(model, z[j = 1:n_games], Bin)


### Match Winning Constraint ###
### R is player ratings, R[i][k] is player i for position k ###
### C is the opposing team rating, C[j] is the rating of the opposing team for game j ###


# for i in starting_11
#     @constraint(model, sum(sum(x[i, j, k] for j in 1:n_games) for k in 1:n_positions) >= 15)
# end

for j in 1:n_games
    @constraint(model, sum(sum(R[i, k]*x[i, j, k]*penalty(i, j, x) for i in 1:n_players) for k in 1:n_positions)/11 >= T_mult*C[j, 2]*z[j])
    # @constraint(model, sum(sum(R[i, k]*x[i, j, k]*penalty(i, j, x) for i in 1:n_players) for k in 1:n_positions)/11 >= 85*z[j])
    @constraint(model, sum(sum(R[i, k]*x[i, j, k]*penalty(i, j, x) for i in 1:n_players) for k in 1:n_positions)/11 >= 75)
end

### Player Constraints ###

### Each player can only play at most once in a game ###
for j in 1:n_games
    for i in 1:n_players
        @constraint(model, sum(x[i, j, k] for k in 1:n_positions) <= 1)
    end
end

### Can only have 11 players in game ###
for j in 1:n_games
    @constraint(model, sum(sum(x[i, j, k] for k in 1:n_positions) for i in 1:n_players) == 11)

    ### 1 GK per game ###
    @constraint(model, sum(x[i, j, pos_to_ind[GK]] for i in 1:n_players) == 1)

    ### 4 Defenders per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in defenders) == 4)

    ### 3 - 5 Midfielders per game ### 
    @constraint(model, 3 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in midfielders) <= 5)

    ### 1 - 3 Forwards per game ###
    @constraint(model, 1 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in forwards) <= 3)

    # ### 2 CBs per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in center_backs) == 2)

    # ### 1 RB per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in right_backs) == 1)

    # ### 1 LB per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in left_backs) == 1)

    # ### 0 - 2 CDMs per game ###
    @constraint(model, 0 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in cdms) <= 2)

    # ### 0 - 3 CMs per game ###
    @constraint(model, 0 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in cms) <= 3)

    # ### 0 - 3 CAMS per game ###
    @constraint(model, 0 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in cams) <= 3)

    # ### 0 - 1 LW, LM per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in left_wingers) <= 1)

    # ### 0 - 1 RW, RM per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in right_wingers) <= 1)

    # ### 1 ST per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in strikers) == 1)
end

### Objective Function ###
@objective(model, Max, sum(z[j] for j in 1:n_games))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-03


z[1] + z[2] + z[3] + z[4] + z[5] + z[6] + z[7] + z[8] + z[9] + z[10] + z[11] + z[12] + z[13] + z[14] + z[15] + z[16] + z[17] + z[18] + z[19] + z[20] + z[21] + z[22] + z[23] + z[24] + z[25] + z[26] + z[27] + z[28] + z[29] + z[30] + z[31] + z[32] + z[33] + z[34] + z[35] + z[36] + z[37] + z[38]

In [224]:
optimize!(model)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1520 rows, 24852 columns and 147060 nonzeros
Model fingerprint: 0x5ef38f10
Variable types: 190 continuous, 24662 integer (24662 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 8e+01]
Presolve removed 1514 rows and 24840 columns
Presolve time: 0.11s
Presolved: 6 rows, 12 columns, 35 nonzeros
Variable types: 0 continuous, 12 integer (11 binary)
Found heuristic solution: objective 30.0000000

Explored 0 nodes (0 simplex iterations) in 0.15 seconds (0.08 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 30 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+01, best bound 3.000000000000e+01, gap 0.0000%

User-callback calls 654, time in user-callback 0.00 sec


In [225]:
opt_x = value.(x)
function matrix_to_lineup(x, j)
    player_to_pos = Dict()
    for i in 1:n_players
        for k in 1:n_positions
            cur_val = x[i, j, k]
            if cur_val == 1
                player_name = liverpool_data[i, "short_name"]
                # println(player_name, i)
                player_pos = ind_to_pos[k]
                player_to_pos[player_name] = player_pos
            end
        end
    end
    return player_to_pos
end

println(matrix_to_lineup(opt_x, 1))

Dict{Any, Any}(String31("S. Mané") => "lw", String31("J. Milner") => "cm", String31("M. Salah") => "rw", String31("N. Keïta") => "cm", String31("A. Oxlade-Chamberlain") => "cm", String31("V. van Dijk") => "cb", String31("T. Alexander-Arnold") => "rb", String31("Fabinho") => "cb", String31("A. Robertson") => "lb", String31("Alisson") => "gk", String31("Diogo Jota") => "st")


In [239]:
function every_lineup(x)
    game_to_lineup = Dict()
    for j in 1:n_games
        game_to_lineup[j] = matrix_to_lineup(x, j)
    end
    return game_to_lineup
end

line_ups = every_lineup(opt_x)

Dict{Any, Any} with 38 entries:
  5  => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Milner")=>"cm", …
  16 => Dict{Any, Any}(String31("J. Matip")=>"cb", String31("M. Salah")=>"rw", …
  20 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  35 => Dict{Any, Any}(String31("M. Salah")=>"rw", String31("N. Keïta")=>"cm", …
  12 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  24 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Matip")=>"cb", S…
  28 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Milner")=>"cm", …
  8  => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Milner")=>"cm", …
  17 => Dict{Any, Any}(String31("M. Salah")=>"rw", String31("N. Keïta")=>"cm", …
  30 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Milner")=>"cm", …
  1  => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Milner")=>"cm", …
  19 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Milner")=>"cm"

In [236]:
line_ups[36]

Dict{Any, Any} with 11 entries:
  String31("S. Mané")               => "lw"
  String31("J. Milner")             => "cm"
  String31("M. Salah")              => "rw"
  String31("N. Keïta")              => "cm"
  String31("A. Oxlade-Chamberlain") => "cm"
  String31("V. van Dijk")           => "cb"
  String31("T. Alexander-Arnold")   => "rb"
  String31("Fabinho")               => "cb"
  String31("A. Robertson")          => "lb"
  String31("Alisson")               => "gk"
  String31("Diogo Jota")            => "st"

In [240]:
function dict_to_df(lineup_dict)
    player_names, player_pos = [], []
    for (key, value) in lineup_dict
        push!(player_names, key)
        push!(player_pos, value)
    end
    df = DataFrame(player_name = player_names, position = player_pos)
    return df
end
dict_to_df(line_ups[1])

Row,player_name,position
,Any,Any
1,S. Mané,lw
2,J. Milner,cm
3,M. Salah,rw
4,N. Keïta,cm
5,A. Oxlade-Chamberlain,cm
6,V. van Dijk,cb
7,T. Alexander-Arnold,rb
8,Fabinho,cb
9,A. Robertson,lb


In [241]:
fixtures = C[!, "Team"]
XLSX.openxlsx("mult_penalty_beatavg.xlsx", mode="w") do xf
    XLSX.rename!(xf[1], fixtures[1] * "_1")
    teams = Set([fixtures[1]])
    sheet = xf[fixtures[1] * "_1"]
    cur_lineup = line_ups[1]
    df = dict_to_df(cur_lineup)
    for r in 1:size(df,1), c in 1:size(df,2)
        sheet[XLSX.CellRef(r , c )] = df[r,c]
    end
    for game in 2:n_games
        cur_team = fixtures[game]
        if cur_team in teams
            sheet_name = cur_team * "_2"
        else
            sheet_name = cur_team * "_1"
            push!(teams, cur_team)
        end
        XLSX.addsheet!(xf, sheet_name)
        sheet = xf[sheet_name]
        cur_lineup = line_ups[game]
        df = dict_to_df(cur_lineup)
        for r in 1:size(df,1), c in 1:size(df,2)
            sheet[XLSX.CellRef(r , c )] = df[r,c]
        end
    end
end